In [28]:
!python3 --version
#!python --version
#!jupyter --version

import inspect

Python 3.10.6


# Gaussian Elimination in plan python

In [24]:
import gauss as G
import algebra as A
import elementary_matrix as E
import properties as P

In [45]:
dir(G)
print(inspect.signature(G.add))
print(inspect.getsource(G.add))
#inspect.findsource(G.add)
print(inspect.signature.__doc__)
print(inspect.get_annotations(G.add))

(M1: list[list[fractions.Fraction | float | int]], M2: list[list[fractions.Fraction | float | int]]) -> tuple[list[list[fractions.Fraction | float | int]], str]
def add(M1: M, M2: M) -> tuple[M, str]:
    """element-wise addition of two matrices. in functional style.

    fp alternative approch using map:
    >>> from operator import add
    >>> list( map(add, list1, list2) )
    [5, 7, 9]
    """

    if len(M1) != len(M2):
        return ((), "Rank is not identified")
    return ([sum(x) for x in zip(M1, M2)], "")

Get a signature object for the passed callable.
{'M1': list[list[fractions.Fraction | float | int]], 'M2': list[list[fractions.Fraction | float | int]], 'return': tuple[list[list[fractions.Fraction | float | int]], str]}


In [30]:
dir(inspect)

['ArgInfo',
 'ArgSpec',
 'Arguments',
 'Attribute',
 'BlockFinder',
 'BoundArguments',
 'CORO_CLOSED',
 'CORO_CREATED',
 'CORO_RUNNING',
 'CORO_SUSPENDED',
 'CO_ASYNC_GENERATOR',
 'CO_COROUTINE',
 'CO_GENERATOR',
 'CO_ITERABLE_COROUTINE',
 'CO_NESTED',
 'CO_NEWLOCALS',
 'CO_NOFREE',
 'CO_OPTIMIZED',
 'CO_VARARGS',
 'CO_VARKEYWORDS',
 'ClassFoundException',
 'ClosureVars',
 'EndOfBlock',
 'FrameInfo',
 'FullArgSpec',
 'GEN_CLOSED',
 'GEN_CREATED',
 'GEN_RUNNING',
 'GEN_SUSPENDED',
 'OrderedDict',
 'Parameter',
 'Signature',
 'TPFLAGS_IS_ABSTRACT',
 'Traceback',
 '_ClassFinder',
 '_ClassMethodWrapper',
 '_KEYWORD_ONLY',
 '_MethodWrapper',
 '_NonUserDefinedCallables',
 '_PARAM_NAME_MAPPING',
 '_POSITIONAL_ONLY',
 '_POSITIONAL_OR_KEYWORD',
 '_ParameterKind',
 '_VAR_KEYWORD',
 '_VAR_POSITIONAL',
 '_WrapperDescriptor',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_check_class',
 '_check_instance',
 '_empty'

In [39]:
ms = [ [[1,2],[2,3]], [ [1,2,3], [2,3,4] ]]
list(map(P.show, ms))

1	2
2	3
1	2	3
2	3	4


[None, None]

In [41]:
P.show([[1,2], [2,3]])

1	2
2	3


## Functional Paradigm / Category Theory

- `map` applies a function on `a` to a list of `a`s:
```map(increment, [1,2,3]) -> [increment(2), increment(3), increment(4)] -> [2,3,4]```
- `reduce` / `fold` combines two elements of a list and accumulates the result
  `fold(add, [1,2,3]) -> add(add(1,2),3)` -> 6

## Improvements / Refactoring

- The gauss function is quite big and deeply nested. It would be nice to have separate functions matching the steps in the algorithm.
- Implement different approaches, run performance tests and analyze. Maybe some heuristics can help to speed up unlucky cases (e.g. pivots all the way to the right)
- What about correctness?
- Make precise drawings of the data structures and operations
- Check properties and assert they hold within the algorithm.
- Represent `Matrix` as a class with properties and functions